In [4]:
reset()
%display latex
Partitions.options.latex="list"

# Demo: quantum toroidal $\mathfrak{gl}_1$ algebra

In [5]:
load("setup.sage")
load("quantum_toroidal.sage")

### General setup

The underlying ring for the algebra is $\mathbb{Q}(x,y)$. For some applications, we require $x^{1/2}$ and $y^{1/2}$, so we pre-emptively square those variables.

In [27]:
R.<x,y> = LaurentPolynomialRing(ZZ)

The command `QuantumToroidalAlgebra(x, y)` creates the algebra $U_{x,y}(\widehat{\widehat{\mathfrak{gl}}}_1)$ over the fraction field of wherever $x$ and $y$ live.

In [28]:
QTA = QuantumToroidalAlgebra(x, y)
QTA

Quantum toroidal algebra (t1=x and t2=y)

Ask Sage what type of object `QTA` is.

In [29]:
QTA.category()

Category of filtered algebras with basis over Fraction Field of Multivariate Polynomial Ring in x, y over Integer Ring

Unfortunately the coproduct is not implemented yet.

### Presentation as elliptic Hall algebra

We use the presentation of $U_{x,y}(\widehat{\widehat{\mathfrak{gl}}}_1)$ as the *elliptic Hall algebra*. This means it is generated by elements $e_{(a,b)}$ and $K_{(a,b)}$ for $(a,b) \in \mathbb{Z}^2$, with the following relations.

1. $K_{(a,b)} K_{(a',b')} = K_{(a+a', b+b')}$ and $e_{(0,0)} = K_{(0,0)} = 1$.

In [16]:
QTA.K(2,3) * QTA.K(-1,4), QTA.e(0,0), QTA.K(0,0)

(K(1,7), u, u)

2. If $\vec a$ and $\vec b$ are collinear, then
$$ [e_{\vec a}, e_{\vec b}] = \delta_{\vec a+\vec b} \frac{K_{\vec a}^{-1} - K_{\vec a}}{n_{\mathrm{deg}(\vec a)}} $$
where $\mathrm{deg}(\vec a) = \mathrm{gcd}(a_1,a_2)$ and $n_d = (1 - x^d)(1 - y^d)(1 - (xy)^{-d})/d$ (implemented as `QTA.n(d)`).

This means that, for each slope $s \in \mathbb{Q}$, the lattice points of slope $s$ form a quantum $\widehat{\mathfrak{gl}}_1$-subalgebra.

In [18]:
def bracket(a, b):
    return a*b - b*a

pretty( bracket(QTA.e(1,2), QTA.e(-1,-2)) )

(-x*y/((x*y-1)*(x-1)*(y-1)))*K(1,2) + (x*y/((x*y-1)*(x-1)*(y-1)))*K(-1,-2)

In [19]:
bracket(QTA.e(1,2), QTA.e(-2,-4))

0

3. If $\vec a$ and $\vec b$ are such that the triangle formed by $\vec 0$, $\vec a$, $\vec a + \vec b$ has no interior lattice points (and $\deg \vec a = 1$), then
$$ [e_{\vec a}, e_{\vec b}] = -\epsilon_{\vec a, \vec b} K_{\alpha(\vec a, \vec b)} \frac{\Psi_{\vec a + \vec b}}{n_1} $$
for a sign $\epsilon_{\vec a, \vec b}$ and some function $\alpha(\vec a, \vec b)$. See `quantum_toroidal.pdf` for details. The elements $\Psi$ are defined by the (perhaps familiar) plethystic expression
$$ \sum_{k=0}^\infty \Psi_{k \vec c} z^k = \exp\left(\sum_{m=1}^\infty n_m e_{m\vec c} z^m\right) $$
for $\deg \vec c = 1$. They are implemented as `QTA.Psi(a1,a2)`.

In [20]:
bracket(QTA.e(1,1), QTA.e(0,1))

-e(1,2)

The third relation does *not* mean that $e_{\vec a}$ and $e_{\vec b}$ commute if the triangle has interior lattice points. Rather, there is a recursive algorithm to pick a sequence of interior lattice points sub-dividing the triangle into "primitive" ones without interior lattice points. See `quantum_toroidal.pdf` for details.

In [21]:
pretty( bracket(QTA.e(2,0), QTA.e(1,2)) )

(-(x^2*y^2-x*y+x+y)/(x*y))*e(3,2) + (-(x*y-1)*(x-1)*(y-1)/(x*y))*e(2,1)*e(1,1)

Results are always *normal ordered*: each monomial is a product of terms $e_{(a,b)}$ ordered by increasing slope $b/a$, and then by decreasing degree within each slope. All central elements $K_{(a,b)}$ appear first.

In [22]:
QTA.e(2,0) * QTA.e(1,2) * QTA.K(-3,-2)

K(-3,-2)*e(2,0)*e(1,2)

In [23]:
pretty( QTA.e(1,2) * QTA.e(2,0) * QTA.K(-3,-2) )

((x^2*y^2-x*y+x+y)/(x*y))*K(-3,-2)*e(3,2) + K(-3,-2)*e(2,0)*e(1,2) + ((x*y-1)*(x-1)*(y-1)/(x*y))*K(-3,-2)*e(2,1)*e(1,1)

### Drinfeld presentation

There is an alternate presentation of $U_{x,y}(\widehat{\widehat{\mathfrak{gl}}}_1)$ in terms of fields $e(z) = \sum_{i \in \mathbb{Z}} e_i z^{-i}$, $f(z) = \sum_{i \in \mathbb{Z}} f_i z^{-i}$, and $\psi^\pm(z) = \sum_{j \ge 0} \psi_{\pm j} z^{\mp j}$, with relations like

$$ \begin{align*}
e(z) e(w) (z - xw)(z - yw)(z - (xy)^{-1} w) &= -e(w) e(z) (w - xz) (w - yz) (w - (xy)^{-1} z) \\
\psi^\pm(z) e(w) (z - xw)(z - yw)(z - (xy)^{-1} w) &= -e(w) \psi^\pm(z) (w - xz) (w - yz) (w - (xy)^{-1} z) \\
[e(z), f(w)] &= \frac{\delta(z/w)}{(1 - x)(1 - y)(1 - (xy)^{-1})} (\psi^+(w) - \psi^-(z))
\end{align*} $$

The identification with our EHA presentation is that $e_i = e_{(-1,i)}$, $f_i = f_{(1,i)}$, and $\psi_j = K_{(0,-j)} \Psi_{(0,j)}$.

We can check this identification (for the first two given relations; the third is sort of obvious).

In [33]:
ee = lambda r: QTA.e(-1, r)
ff = lambda r: QTA.e(1, r)
ppsi = lambda r: QTA.K(0,-r)*QTA.Psi(0, r)

q1, q2, q3 = x, y, (x*y)^-1
all( ee(r+3)*ee(s) - (q1+q2+q3)*ee(r+2)*ee(s+1) + (q1*q2 + q1*q3 + q2*q3)*ee(r+1)*ee(s+2) - ee(r)*ee(s+3) ==
       -(ee(s+3)*ee(r) - (q1+q2+q3)*ee(s+2)*ee(r+1) + (q1*q2 + q1*q3 + q2*q3)*ee(s+1)*ee(r+2) - ee(s)*ee(r+3)) and
     ppsi(r+3)*ee(s) - (q1+q2+q3)*ppsi(r+2)*ee(s+1) + (q1*q2 + q1*q3 + q2*q3)*ppsi(r+1)*ee(s+2) - ppsi(r)*ee(s+3) ==
       -(ee(s+3)*ppsi(r) - (q1+q2+q3)*ee(s+2)*ppsi(r+1) + (q1*q2 + q1*q3 + q2*q3)*ee(s+1)*ppsi(r+2) - ee(s)*ppsi(r+3))
     for r in range(4) for s in range(4) )

True

### The level-$(0,1)$ Fock representation

There is a geometric realization of $U_q(\widehat{\widehat{\mathfrak{gl}}}_1)$ via $1$-step correspondences in the (equivariant) K-theory of $\mathrm{Hilb}(\mathbb{C}^2)$. Therefore it has a natural action on

$$ K_{\mathsf{T}}(\mathrm{Hilb}(\mathbb{C}^2)) $$

which is isomorphic to the ring of symmetric functions over $K_{\mathsf{T}}(\mathrm{pt}) = \mathbb{Z}[x^\pm, y^\pm]$. Here $\mathsf{T} = (\mathbb{C}^\times)^2$ is the natural torus scaling $\mathbb{C}^2$.

Some square roots are involved here, so we re-define all our quantum toroidal algebra to use variables $x^2$ and $y^2$ instead of $x$ and $y$.

In [6]:
R.<x,y> = LaurentPolynomialRing(ZZ)
x, y = x^2, y^2
QTA = QuantumToroidalAlgebra(x, y)
sym = SymmetricFunctions(R.fraction_field())
sym

Symmetric Functions over Fraction Field of Multivariate Polynomial Ring in x, y over Integer Ring

The action is uniquely characterized using the algebra relations, and the following explicit descriptions.

1. Elements $e_{(m,0)}$ of the "horizontal" subalgebra act as a Heisenberg algebra, namely for $m > 0$
$$ \begin{align*}
e_{(m,0)} &= m \frac{\partial}{\partial p_m} \\
e_{(-m,0)} &= \frac{(-xy)^{m/2}}{(1 - x^m)(1 - y^m)} p_m
\end{align*} $$
where $\{p_k\}$ is the power sum basis for the ring of symmetric functions.

In [48]:
p = sym.powersum()
QTA.e(1,0) * p[1,1]

2*p[1]

In [49]:
QTA.e(-1,0) * p[1]

((-x*y)/(x^2*y^2-x^2-y^2+1))*p[1, 1]

2. Elements $e_{(0,m)}$ of the "vertical" subalgebra commute and act diagonally in the fixed point basis (Macdonald polynomials $\tilde H_\lambda$ in Haiman's normalization) with eigenvalues related to the tautological bundles at fixed points. Explicitly,
$$ e_{(0,k)} \tilde H_\lambda = \mathrm{sign}(k) \left(\frac{1}{1 - x^k} \sum_{i=1}^\infty x^{k\lambda_i} y^{k(i-1)}\right) \tilde H_\lambda. $$

In [7]:
mcd = sym.macdonald(q=x, t=y)
mcd_ht = mcd.Ht()
mcd_ht

Symmetric Functions over Fraction Field of Multivariate Polynomial Ring in x, y over Integer Ring in the Macdonald Ht with q=x^2 and t=y^2 basis

In [8]:
QTA.e(0,1) * mcd_ht[2,1]

((-x^4*y^2-x^2*y^4+x^4+x^2*y^2+y^4)/(x^2*y^2-x^2-y^2+1))*McdHt[2, 1]

Actions of the remaining generators $e_{(a,b)}$ are uniquely determined by using relation (3) of the quantum toroidal algebra to reduce to actions of the horizontal and vertical generators. See `quantum_toroidal.pdf` for details.

**Remark**. It is a very useful and non-trivial test of all the formulas above to verify that the action really is an algebra homomorphism.

**Warning**: this may take a long time to execute.

In [11]:
def test_alg_hom(a, b, n):
    return QTA.e(a) * (QTA.e(b) * mcd_ht[n]) == (QTA.e(a) * QTA.e(b)) * mcd_ht[n]

import random
def random_Z2_point_within(r):
    return (ZZ^2)((random.randint(-r,r), random.randint(-r,r)))

all( test_alg_hom(random_Z2_point_within(4), random_Z2_point_within(4), Partitions(5).random_element())
     for _ in range(10) )

True

### An application: wall operators for $\mathrm{Hilb}(\mathbb{C}^2)$

The q-difference operators in K-theoretic quasimap theory of Nakajima quiver varieties are products of so-called *wall operators* for walls in some affine hyperplane arrangement associated to the quiver. These wall operators have an explicit characterization in terms of the associated quantum group. For $\mathrm{Hilb}(\mathbb{C}^2)$ this is exactly our quantum toroidal algebra.

Create the rank-$1$ instanton moduli space.

In [52]:
load("instantons.sage")
R.<t1,t2,z,q> = LaurentPolynomialRing(ZZ)
t1, t2 = t1^2, t2^2
M = Instantons(1, t1, t2)

The command `M.qde_wall(n, w, q, z, B)` returns the wall operator $B_w(z)$ for instanton number $n$ and the wall $w \in \mathbb{Q}$, written in the basis $B$ of the ring of symmetric functions.

For various reasons, the operator $B_w$ is simplest in the basis of K-theoretic stable envelopes of the same slope $w$.

In [54]:
M.walls(3)

[0, 1/3, 1/2, 2/3]

In [53]:
pretty( M.qde_wall(3, 0, q, z, M.stab(0)) )

[                                                    -(t1*t2*z^3 - 1)*(t1*t2 - z)*(z + 1)*(z - 1)/((t1^2*t2^2*z^2 + t1*t2*z + 1)*(t1*t2*z + 1)*(t1*t2*z - 1)^3*t1*t2)                                                         -(t1*t2 - z)*(t1*t2 + 1)*(t1*t2 - 1)*(z + 1)*(z - 1)*z/((t1^2*t2^2*z^2 + t1*t2*z + 1)*(t1*t2*z - 1)^3*t1*t2)                                       -(t1^3*t2^3*z - 1)*(t1*t2 - z)*(t1*t2 + 1)*(t1*t2 - 1)*z^2/((t1^2*t2^2*z^2 + t1*t2*z + 1)*(t1*t2*z + 1)*(t1*t2*z - 1)^3*t1*t2)]
[                                                    -(t1*t2 - z)*(t1*t2 + 1)*(t1*t2 - 1)*(z + 1)*(z - 1)*z/((t1^2*t2^2*z^2 + t1*t2*z + 1)*(t1*t2*z - 1)^3*t1^3*t2^3) -(t1^4*t2^4*z^2 - t1^3*t2^3*z^3 + t1^2*t2^2*z^4 - 2*t1^2*t2^2*z^2 + t1^2*t2^2 - t1*t2*z + z^2)*(t1*t2 - z)/((t1^2*t2^2*z^2 + t1*t2*z + 1)*(t1*t2*z - 1)^3*t1^3*t2^3)                                                         -(t1*t2 - z)*(t1*t2 + 1)*(t1*t2 - 1)*(z + 1)*(z - 1)*z/((t1^2*t2^2*z^2 + t1*t2*z + 1)*(t1*t2*z - 1)^3*t1*t2)]
[                                  -(t1^3*t2^3*z - 1)*(t1*t2 - z)*(t1*t2 + 1)*(t1*t2 - 1)*z^2/((t1^2*t2^2*z^2 + t1*t2*z + 1)*(t1*t2*z + 1)*(t1*t2*z - 1)^3*t1^5*t2^5)                                                     -(t1*t2 - z)*(t1*t2 + 1)*(t1*t2 - 1)*(z + 1)*(z - 1)*z/((t1^2*t2^2*z^2 + t1*t2*z + 1)*(t1*t2*z - 1)^3*t1^3*t2^3)                                                     -(t1*t2*z^3 - 1)*(t1*t2 - z)*(z + 1)*(z - 1)/((t1^2*t2^2*z^2 + t1*t2*z + 1)*(t1*t2*z + 1)*(t1*t2*z - 1)^3*t1*t2)]

In [55]:
pretty( M.qde_wall(3, 1/3, q, z, M.stab(1/3)) )

[(t1^6*t2^6*z^3 - q*t1^3*t2^3 - t1^2*t2^2*z^3 + z^3)/((t1^3*t2^3*z^3 - q)*t1^3*t2^3)                             -(t1*t2 + 1)*(t1*t2 - 1)*z^3/((t1^3*t2^3*z^3 - q)*t2^2)                          -(t1*t2 + 1)*(t1*t2 - 1)*t1^3*z^3/((t1^3*t2^3*z^3 - q)*t2)]
[                       -(t1*t2 + 1)*(t1*t2 - 1)*z^3/((t1^3*t2^3*z^3 - q)*t1^4*t2^2)         (t1^4*t2^4*z^3 - t1^2*t2^2*z^3 - q*t1*t2 + z^3)/((t1^3*t2^3*z^3 - q)*t1*t2)                               -(t1*t2 + 1)*(t1*t2 - 1)*t1^2*z^3/(t1^3*t2^3*z^3 - q)]
[                         -(t1*t2 + 1)*(t1*t2 - 1)*z^3/((t1^3*t2^3*z^3 - q)*t1^5*t2)                             -(t1*t2 + 1)*(t1*t2 - 1)*z^3/((t1^3*t2^3*z^3 - q)*t1^2)                                                 (t1*t2*z^3 - q)/(t1^3*t2^3*z^3 - q)]

In [56]:
pretty( M.qde_wall(3, 1/2, q, z, M.stab(1/2)) )

[(t1^4*t2^4*z^2 - t1^2*t2^2*z^2 - q*t1^2*t2^2 + z^2)/((t1^2*t2^2*z^2 - q)*t1^2*t2^2)                                                                                   0                         (t1*t2 + 1)*(t1*t2 - 1)*t1^4*z^2/((t1^2*t2^2*z^2 - q)*t2^2)]
[                                                                                  0                                                                                   1                                                                                   0]
[                             (t1*t2 + 1)*(t1*t2 - 1)*z^2/((t1^2*t2^2*z^2 - q)*t1^6)                                                                                   0                                                       (z^2 - q)/(t1^2*t2^2*z^2 - q)]